In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
from datasets import load_dataset
import torch

max_seq_length=2048
dtype=None
load_in_4bit =True

In [ ]:
fourbit_models=[
    "unsloth/mistral-7b-v0-bnb-4bit",
    "unsloth/mistral-7b-listruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/phi-3-mini-4k-instruct",
    "unsloth/minstral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/sloar-10.7b-bnb-4bit"
    ]
model,tokenizer=FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
     )

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task
Write a response that appropriately completes the request in korean language.

### Instruction:
{}
### Input:
{}

### Response:
{}
"""
#데이터들 text들로 변환
EOS_TOKEN =tokenizer.eos_token
def formatting_prompt_func(examples):
  instructions = examples["instruction"]
  inputs=examples["input"]
  outputs=examples["output"]
  texts=[]
  for instructions,inputs,outputs in zip(instructions,inputs,outputs):
    text=alpaca_prompt.format(instructions,inputs,outputs)+EOS_TOKEN
    texts.append(text)
  return {"text":texts,}
  pass

In [ ]:
import pandas as pd
from datasets import Dataset
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항을 가져옵니다.
    inputs = examples["input"]  # 입력값 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction,input,output in zip(instructions, inputs , outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction,input, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }

#사용할 csv데이터 가져오는 코드
df = pd.read_csv('/content/drive/MyDrive/sample/law_usloth_data.csv')

# Convert the DataFrame to a Hugging Face dataset
dataset = Dataset.from_pandas(df)

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

In [ ]:
from unsloth import FastLanguageModel

# 💡 모델을 로드해서 사용할 준비하는 코드에요
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # ✅ 오타 수정됨
    bias="none",
    use_gradient_checkpointing = "unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

In [ ]:
import os
os.environ["WANDB_DISABLED"]="True"

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

#실제로 모델을 학습시키는 코드입니다.
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        max_steps = 200,#이게 에포크에요
        learning_rate = 2e-4,
        lr_scheduler_type = "cosine",
        weight_decay = 0.01,
        fp16 = False,
        bf16 = True,
        logging_steps = 10,
        save_steps = 200,
        save_total_limit = 2,
        seed = 42,
        optim = "adamw_8bit",
        output_dir = "output",
        report_to = "none"
    )
)
trainer_state=trainer.train()

In [ ]:
# 추론용으로 변환
model = FastLanguageModel.for_inference(model)

# --- 법률 QA Prompt 템플릿 ---
def law_prompt(instruction, question, context=""):
    return f"""### instruction:
{instruction}

### question:
{question}

### 답변:
"""

# 입력데이터에요
instruction = "법률 질의응답을 작성할 때, 법 조항의 핵심 내용을 바탕으로 간결하게 답변할 것"
question = "어떤 창작물은 저작권 보호를 받을 수 없다고 하는데, 그 이유는 무엇인가요? 예시를 들어 설명해주세요."

prompt = law_prompt(instruction, question)

#받은 입력데이터를 바탕으로 모델이 출력을 만들어내게 하는 코드에요
inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)

# 결과 출력
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print("=== 결과 ===")
print(answer.split("### 답변:")[-1].strip())

In [ ]:
#입력데이터에요
instruction = "법률 질의응답을 작성할 때, 법 조항의 핵심 내용을 바탕으로 간결하게 답변할 것"
question = "내가 만약 라이센스가 등록된 제품을 무단으로 사용하였을때 어떻게되?"

prompt = law_prompt(instruction, question)

# Tokenize & Generate
inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)

# 결과 출력
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print("=== 결과 ===")
print(answer.split("### 답변:")[-1].strip())

In [ ]:
instruction = "법률 질의응답을 작성할 때, 법 조항의 핵심 내용을 바탕으로 간결하게 답변할 것"
question = "그렇다면 언제 형사상으로도 처벌받을 수 있으며, 법적 조치가 내려질 수 있어?"

prompt = law_prompt(instruction, question)

# Tokenize & Generate
inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)

# 결과 출력
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [ ]:
print("=== 결과 ===")
print(answer.split("### 답변:")[-1].strip())

In [ ]:
#여기서 부터는 허깅페이스의 들어가서 모델 업로드 하는 코드인데 아직 만족스럽지가 않아서 안하고 있어요
from huggingface_hub import login

# 1. 허깅페이스 로그인
huggingface_token = "github에서 토근은 가리라고 하네요"
huggingface_repo = "kim0924/Llama-3-law-8b-version3-merged"
login(token=huggingface_token)

# 2. 모델과 토크나이저를 push_to_hub
model.push_to_hub(
    huggingface_repo,
    use_temp_dir=False,
    private=True,  # 공개 여부 선택
)

tokenizer.push_to_hub(
    huggingface_repo,
    use_temp_dir=False,
)

In [ ]:
from huggingface_hub import create_repo

create_repo(
    repo_id="kim0924/Llama-3-law-8b-version2-gguf",
    token="github에서 토근은 가리라고 하네요",
    repo_type="model",  # 반드시 'model'로 지정
    exist_ok=True       # 이미 있어도 에러 없이 넘어감
)


In [ ]:
huggingface_token="github에서 토근은 가리라고 하네요 "
huggingface_repo="kim0924/Llama-3-law-8b-version3-gguf"
save_method=("merged_16bit")
qantization_method="q8_0"

In [ ]:
model.push_to_hub_gguf(
    huggingface_repo,
    tokenizer,
    token=huggingface_token,
    quantization_method=qantization_method
)

In [ ]:
import os
import pandas as pd
from datasets import Dataset

fileName = "indata_kor"
dataset_Path = "/content/drive/MyDrive/sample"
csv_file_path = os.path.join(dataset_Path, f"{fileName}.csv")

df = pd.read_csv(csv_file_path, encoding="utf-8")
dataset = Dataset.from_pandas(df)

df = df[['instruction', 'output']]
df["input"]=""
dataset = Dataset.from_pandas(df)

from huggingface_hub import login
token_key = "토큰 가리기"
state=login(token=token_key)

dataset.push_to_hub("kim0924/lama3_kim_unslot_data")